# Load Packages

In [1]:
include("printmat.jl")

println4Ps (generic function with 1 method)

# Some Important Types

Julia has many different types of variables: signed integers (like 2 or -5), floating point numbers (2.0 and -5.1), bools (false/true), bitarrays (similar to bools, but with more efficient use of memory), strings ("hello"), Dates (2017-04-23) and many more types. 

The numerical types also comes with subtypes for different precisions, for instance, Float16, Float32 and Float64. Unless you specify otherwise, code like
```
a = 2
b = 2.0
```
gives an Int64 and a Float64 respectively (at least on Windows 7, 64 bit).

## Integers and Floats

In [2]:
a = 2                   #integer, Int (Int64 on most machines)
b = 2.0                 #floating point, (Float64 on most machines)
A = [1;2]
B = [1.0;2.0]

println(typeof(a))
printmat(a)

println(typeof(b))
printmat(b)

println("\n",typeof(A))
printmat(A)

println("\n\n",typeof(B))
printmat(B)

Int64
         2

Float64
     2.000


Array{Int64,1}
         1
         2



Array{Float64,1}
     1.000
     2.000



# Why Use Int When There Are Floats? 

That is, why bother with sometimes using 3 when you could use 3.0 everywhere? Mostly because you cannot use 3.0 everywhere... For instance, you cannot pick out element x[3.0] from a vector. It has to be x[3].

In [3]:
x = [1;10;100;1000]
try 
   y = x[3.0]   
catch    
  println("no, x[3.0] does not work")    
end

no, x[3.0] does not work


## Bools and BitArrays

In [4]:
c = 2 > 1.1
println("c: ",typeof(c))
printmat(c)

C = A .> 1.5
println("\nC: ",typeof(C))
printmat(C)

println("a BitArray is essentially a more economical array version of Bool as seen from typeof(C[1]): ",typeof(C[1]))

c: Bool
      true


C: BitArray{1}
     false
      true

a BitArray is essentially a more economical array version of Bool as seen from typeof(C[1]): Bool


## Char and Strings

In [5]:
t = 'a'                                    #Char, just one letter
println(typeof(t))

txt = "Dogs are nicer than cats."          #String, could be a long novel 
println(typeof(txt))

Char
String


# Calulations with Mixed Types and Converting Types

A calculation like "integer" + "float" works and the type of the result will be a float (the more flexible type). Similarly, "bool" + "integer" will give an integer. These promotion rules make it easy to have mixed types in calculations, and also provide a simple way of converting a variable from one type to another. (There are also an explicit convert() function that might be quicker.)

## Some Calculations with Mixed Types ("promotion")

In [6]:
println(1+2.0)                #integer + Float
println((1.>0) + 2)           #bool + integer

3.0
3


## Converting from Int to Float and vice versa

In [7]:
B_to_int = map(x->round(Int,x),B)        #float -> integer by rounding
println("B after being rounded to Int")
println(typeof(B_to_int))
printmat(B_to_int)


A_to_float = A + 0.0                    #int -> float by adding 0.0
println("\nA after being converted into floats")    #convert(Array{Float64},A) also works
println(typeof(A_to_float))
printmat(A_to_float)

B after being rounded to Int
Array{Int64,1}
         1
         2


A after being converted into floats
Array{Float64,1}
     1.000
     2.000



## Converting from Bools and BitArrays to Int and vice versa

In [8]:
C_to_int = C + 0                         #BitArray -> int by adding 0
println(typeof(C_to_int))                #convert(Array{Int},C) also works
printmat(C_to_int)

D = [1;0;1]                              
D_to_bit = D .> 0                        #int -> BitArray by comparing
println(typeof(D_to_bit))                #convert(BitArray,D) also works 
printmat(D_to_bit)

Array{Int64,1}
         0
         1

BitArray{1}
      true
     false
      true



# Type Instability

Your code will run faster if your variables do not change type in the computations. The next cells illustrate that.

In [9]:
function fn1(n)
    x = 0                    #x starts out as an Int
    for i = 1:n
        x = x + 0.1          #x is changed from Int to a Float when you add 0.1
    end
    return x
end    

function fn2(n)
    x = 0.0                  #x starts out as a Float
    for i = 1:n
        x = x + 0.1          #x remains a float
    end
    return x
end

fn2 (generic function with 1 method)

In [10]:
x = fn1(10)                #a "dry" run makes the subsequent timing results more accurate
x = fn2(10)

@time fn1(1e+6)
@time fn2(1e+6)

println("\nfn2() is much faster and uses much less memory")

  0.094565 seconds (2.05 M allocations: 32.661 MB, 9.11% gc time)
  0.005737 seconds (1.23 k allocations: 58.244 KB)

fn2() is much faster and uses much less memory
